# Deep Learning and Neural Networks
Deep learning is a specific subfield of machine learning: a new take on learning representations from data that puts an emphasis on learning successive layers of increasingly meaningful representations. The deep in deep learning isn’t a reference to any kind of deeper understanding achieved by the approach; rather, it stands for this idea of successive layers of representations. How many layers contribute to a model of the data is called the depth of the model. 

In deep learning, these layered representations are (almost always) learned via models called neural networks, structured in literal layers stacked on top of each other. The term neural network is a reference to neurobiology, but although some of the central concepts in deep learning were originally developed in part by drawing inspiration from our understanding of the brain, **deep-learning models are not models of the brain**. 

The whole biological connection is overblown, and mostly only of interest to journalists and bloggers.  

What do the representations learned by a deep-learning algorithm look like? Let’s examine how a network several layers deep transforms an image of a digit in order to recognize what digit it is.

![Deep Net](img/dnn1.jpg)

The network transforms the digit image into representations that are increasingly different from the original image and increasingly informative about the final result. You can think of a deep network as a multistage information-distillation operation, where information goes through successive filters and comes out increasingly purified.

![Deep Net](img/dnn2.jpg)

To control the output of a neural network, you need to be able to measure how far this output is from what you expected. This is the job of the loss function of the network, also called the objective function. Just like other supervised ML, the loss function takes the predictions of the network and the true target (what you wanted the network to output) and computes a distance score.  

The fundamental trick in deep learning is to use this score as a feedback signal to adjust the value of the weights a little, in a direction that will lower the loss score for the current example (see figure 1.9). This adjustment is the job of the optimizer, which implements what’s called the Backpropagation algorithm: the central algorithm in deep learning. 

![Deep Net](img/dnn3.jpg)

In [2]:
from keras import models
from keras import layers

# Here is a first network architecture
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

ModuleNotFoundError: No module named 'keras'

### 3 additional parts

* A loss function— How the network will be able to measure its performance on the training data, and thus how it will be able to steer itself in the right direction.
* An optimizer— The mechanism through which the network will update itself based on the data it sees and its loss function.
* Metrics to monitor during training and testing— Here, we’ll only care about accuracy (the fraction of the images that were correctly classified).

As you can see, these are the same things we need in other ML.

### What is a Tensor?

DL is nearly synonomous with tensors, because of Tensorflow. Tensors are known to engineers as multidimensional arrays (usually of numbers).  Their algebra works much the same way as matrices.  Scalars (0 dimensional), vectors (1D) and matrices (2D) are just familiar kinds of tensors.  

They are useful because they allow you to pass multiple data points as a single object, and use the rules of linear algebra much like we did with recommenders.  

Groups of same sized grayscale images make 3D tensors.  Groups of video can make 4D tensors, as can groups of color images.  Text can be transformed into a 1D tensor.  Timeseries data can be 2D or more.  They are not hard to get used to.  



### Convolutional networks

Convolutional networks are the workhorse of modern image classification.  They do very well and are getting better. 

In [7]:
# ! pip install keras
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Importantly, a convnet takes as input tensors of shape (image_height, image_width, image_channels) (not including the batch dimension). In the MNIST case, we’ll configure the convnet to process inputs of size (28, 28, 1), which is the format of MNIST images. We’ll do this by passing the argument input_shape=(28, 28, 1) to the first layer.

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


You can see that the output of every Conv2D and MaxPooling2D layer is a 3D tensor of shape (height, width, channels). The width and height dimensions tend to shrink as you go deeper in the network. The number of channels is controlled by the first argument passed to the Conv2D layers (32 or 64).

The next step is to feed the last output tensor (of shape (3, 3, 64)) into a densely connected classifier network like those you’re already familiar with: a stack of Dense layers. These classifiers process vectors, which are 1D, whereas the current output is a 3D tensor. First we have to flatten the 3D outputs to 1D, and then add a few Dense layers on top.

In [9]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

In [11]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 23s 389us/step - loss: 0.1822 - acc: 0.9432
Epoch 2/5
60000/60000 [==============================] - 23s 384us/step - loss: 0.0476 - acc: 0.9853
Epoch 3/5
60000/60000 [==============================] - 23s 388us/step - loss: 0.0326 - acc: 0.9894
Epoch 4/5
60000/60000 [==============================] - 23s 383us/step - loss: 0.0246 - acc: 0.9924
Epoch 5/5
60000/60000 [==============================] - 24s 405us/step - loss: 0.0191 - acc: 0.9938


In [12]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

10000/10000 [==============================] - 1s 120us/step


0.9912

The fundamental difference between a densely connected layer and a convolution layer is this: Dense layers learn global patterns in their input feature space (for example, for a MNIST digit, patterns involving all pixels), whereas convolution layers learn local patterns.  In the case of images, patterns found in small 2D windows of the inputs. In the previous example, these windows were all 3 × 3.

![Deep Net](img/cnn1.jpg)

Visual data convolutions have 2 interesting properties:
1. Translation invariant
2. Spatial hierarchies of patterns

![Cat](img/cat.jpg)

Take a look at (https://playground.tensorflow.org) to play with the visual neural nets.  